In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    
    
        
   # for filename in filenames:

     #   print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install imutils


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
##import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [5]:
from keras.layers import BatchNormalization as bn

In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('')

In [7]:
k=os.walk('../input/facemask/face-mask-detector')

In [8]:
"""%%bash
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str,
default="mask_detector.model",
help="path to output face mask detector model")
args= vars(ap.parse_args())"""


'%%bash\nap = argparse.ArgumentParser()\nap.add_argument("-d", "--dataset", required=True,\nhelp="path to input dataset")\nap.add_argument("-p", "--plot", type=str, default="plot.png",\nhelp="path to output loss/accuracy plot")\nap.add_argument("-m", "--model", type=str,\ndefault="mask_detector.model",\nhelp="path to output face mask detector model")\nargs= vars(ap.parse_args())'

In [9]:
INIT_LR = 3e-4
EPOCHS = 30
BS = 251

In [10]:
imgpt=list(paths.list_images(('../input/outputofnoout/new-outputs-here/')))

In [11]:
data2=[]
labels2=[]

In [12]:
for i in imgpt:
    label="with_mask"
    image=load_img(i,target_size=(128,128))
    image=img_to_array(image)
    image=preprocess_input(image)
    data2.append(image)
    labels2.append(label)
data2=np.array(data2,dtype="float32")
labels2=np.array(labels2)


In [13]:
#non_face=list(paths.list_images('../input/natural-images/natural_images/car/'))+list(paths.list_images('../input/natural-images/natural_images/dog/'))

In [14]:
#label

In [15]:
#data3=[]
#labels3=[]

In [16]:
"""for i in non_face:
    label="no_face"
    image=load_img(i,target_size=(128,128))
    image=img_to_array(image)
    image=preprocess_input(image)
    data3.append(image)
    labels3.append(label)
data3=np.array(data3,dtype="float32")
labels3=np.array(labels3)"""

'for i in non_face:\n    label="no_face"\n    image=load_img(i,target_size=(128,128))\n    image=img_to_array(image)\n    image=preprocess_input(image)\n    data3.append(image)\n    labels3.append(label)\ndata3=np.array(data3,dtype="float32")\nlabels3=np.array(labels3)'

In [17]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(('../input/facemask/face-mask-detector/dataset/without_mask/')))
data1 = []
labels1 = []
# loop over the image paths
for imagePath in imagePaths:
    
# extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
# load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(128, 128))
    image = img_to_array(image)
    image = preprocess_input(image)
# update the data and labels lists, respectively
    data1.append(image)
    labels1.append(label)
# convert the data and labels to NumPy arrays
data1 = np.array(data1, dtype="float32")
labels1 = np.array(labels1)

[INFO] loading images...


In [18]:
labels=np.concatenate((labels2,labels1))

In [19]:
labels.shape

(1255,)

In [20]:
data=np.concatenate((data2,data1))

In [21]:
data.shape

(1255, 128, 128, 3)

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
#labels[2924]

In [24]:
#lab=le.fit_transform(labels)

In [25]:
#labels=to_categorical(lab)

In [26]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels
	,test_size=0.20, stratify=labels)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
LabelBinarizer()

In [ ]:
#k=tf.keras.applications.ResNet152V2()

In [ ]:
#len(k.layers)

In [ ]:
#fine_tune_at=486

In [ ]:
#k.output

In [35]:
 pretrained_model=tf.keras.applications.ResNet152V2(include_top=False,input_shape=(128,128,3),pooling='avg')

In [ ]:
model=tf.keras.Sequential([
   pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2,activation='softmax')])


In [38]:
pretrained_model.trainable=False

In [37]:
inp=tf.keras.layers.Input(shape=[128,128,3])
x=pretrained_model(inp,training=False)
#x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(1024,activation='relu')(x)
out=tf.keras.layers.Dense(2,activation='softmax')(x)
model1=tf.keras.Model(inp,out)

In [ ]:
#for layer in pretrained_model.layers[:fine_tune_at]:
 #   layer.trainable=False

In [39]:
model1.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
resnet152v2 (Functional)     (None, 2048)              58331648  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 60,431,874
Trainable params: 2,100,226
Non-trainable params: 58,331,648
_________________________________________________________________


In [ ]:
#round(model.optimizer.lr.numpy(),7)

In [ ]:
#tf.keras.backend.clear_session()

In [40]:
def scheduler(epoch,lr):
    if (epoch<=10):
        return lr
    elif (epoch>10 and epoch<=20):
        return lr * tf.math.exp(-0.1)
    elif (epoch>20 and epoch<=30):
        return lr * tf.math.exp(-0.1)*0.1
    else:
        return lr * tf.math.exp(-0.1)*0.001

    
        

In [41]:
callback=tf.keras.callbacks.LearningRateScheduler(scheduler)

In [42]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [43]:
    print("[INFO] compiling model...")
    opt = tf.keras.optimizers.Adam(3e-4)
    model1.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=opt,
	metrics=["accuracy"])
    # train the head of the network
    print("[INFO] training head...")
    H = model1.fit(aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY),callbacks=[callback],
epochs=10)

[INFO] compiling model...
[INFO] training head...
Epoch 1/10
4/4 [==============================] - 7s 2s/step - loss: 0.3505 - accuracy: 0.8416 - val_loss: 0.0940 - val_accuracy: 0.9880
Epoch 2/10
4/4 [==============================] - 4s 1s/step - loss: 0.0578 - accuracy: 0.9841 - val_loss: 0.0981 - val_accuracy: 0.9880
Epoch 3/10
4/4 [==============================] - 4s 991ms/step - loss: 0.0538 - accuracy: 0.9890 - val_loss: 0.0827 - val_accuracy: 0.9880
Epoch 4/10
4/4 [==============================] - 4s 1s/step - loss: 0.0469 - accuracy: 0.9851 - val_loss: 0.0790 - val_accuracy: 0.9880
Epoch 5/10
4/4 [==============================] - 4s 985ms/step - loss: 0.0228 - accuracy: 0.9910 - val_loss: 0.0817 - val_accuracy: 0.9880
Epoch 6/10
4/4 [==============================] - 5s 1s/step - loss: 0.0193 - accuracy: 0.9930 - val_loss: 0.0891 - val_accuracy: 0.9920
Epoch 7/10
4/4 [==============================] - 4s 977ms/step - loss: 0.0114 - accuracy: 0.9960 - val_loss: 0.0911 - val

In [44]:
model1.optimizer.lr.numpy()

0.0003

In [45]:
len(model1.layers)

4

In [46]:
fine_tune_at=516
pretrained_model.trainable=True
for layer in pretrained_model.layers[:fine_tune_at]:
    layer.trainable=False
model1.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(3e-4),
metrics=["accuracy"])
model1.summary()
    # train the head of the network


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
resnet152v2 (Functional)     (None, 2048)              58331648  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 60,431,874
Trainable params: 18,189,314
Non-trainable params: 42,242,560
_________________________________________________________________


In [47]:
model1.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.0003,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [48]:
print("[INFO] training head...")
total_epochs=10+27
H = model1.fit(aug.flow(trainX, trainY, batch_size=251),epochs=total_epochs,
validation_data=(testX, testY),initial_epoch=H.epoch[-1],callbacks=[callback])

[INFO] training head...
Epoch 10/37
4/4 [==============================] - 6s 2s/step - loss: 1.9593 - accuracy: 0.7390 - val_loss: 0.5977 - val_accuracy: 0.5498
Epoch 11/37
4/4 [==============================] - 4s 1s/step - loss: 0.4165 - accuracy: 0.8028 - val_loss: 0.2212 - val_accuracy: 0.9761
Epoch 12/37
4/4 [==============================] - 4s 1s/step - loss: 0.1358 - accuracy: 0.9691 - val_loss: 0.0441 - val_accuracy: 0.9801
Epoch 13/37
4/4 [==============================] - 4s 1s/step - loss: 0.0443 - accuracy: 0.9831 - val_loss: 0.0041 - val_accuracy: 0.9960
Epoch 14/37
4/4 [==============================] - 4s 1s/step - loss: 0.0397 - accuracy: 0.9910 - val_loss: 0.0058 - val_accuracy: 0.9960
Epoch 15/37
4/4 [==============================] - 4s 1s/step - loss: 0.0183 - accuracy: 0.9950 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 16/37
4/4 [==============================] - 4s 1s/step - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 17/3

In [49]:
model1.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 2.2282048e-33,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [ ]:
print("[INFO] training head...")
total_epochs=10+27+5
H = model1.fit(aug.flow(trainX, trainY, batch_size=BS),epochs=total_epochs,
validation_data=(testX, testY),initial_epoch=H.epoch[-2],callbacks=[callback])

In [ ]:
round(model1.optimizer.lr.numpy(),8)

In [ ]:
print("[INFO] training head...")
total_epochs=10+27+5+5
H = model1.fit(aug.flow(trainX, trainY, batch_size=BS),epochs=total_epochs,
validation_data=(testX, testY),initial_epoch=H.epoch[-3],callbacks=[callback])

In [ ]:
model1.optimizer.get_config()

In [ ]:
fine_tune_at=496
pretrained_model.trainable=True
bn.trainable=False
for layer in pretrained_model.layers[:fine_tune_at]:
    layer.trainable=False
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),
metrics=["accuracy"])
model.summary()

In [ ]:
model.optimizer.get_config()

In [ ]:
print("[INFO] training head...")
total_epochs=10+20 +15
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),epochs=total_epochs,
validation_data=(testX, testY),initial_epoch=H.epoch[-2],callbacks=[callback])

In [ ]:
model.optimizer.get_config()

In [ ]:
tf.keras.models.save_model(model1,'./newone_with50larsandlearningratetuning',save_format='h5')

In [ ]:
pt1= list(paths.list_images('../input/3classmodelnewevaluation/new_dataset/with_mask/')) + list(paths.list_images('../input/3classmodelnewevaluation/new_dataset/without_mask'))
datae=[]
labelse=[]
for i in pt1:
    label= i.split(os.path.sep)[-2]
    image=load_img(i,target_size=(128,128))
    image=img_to_array(image)
    image=preprocess_input(image)
    datae.append(image)
    labelse.append(label)

In [ ]:
datae=np.array(datae,dtype="float32")
labelse=np.array(labelse)
le=LabelEncoder()
labelse=le.fit_transform(labelse)
labelse=to_categorical(labelse)
model.evaluate(datae,labelse)
predidxs=model.predict(datae,batch_size=512)
predidxs=np.argmax(predidxs,axis=1)
print(classification_report(labelse.argmax(axis=1),predidxs,target_names=le.classes_))

## mtcnn-tf section

In [ ]:
!pip install mtcnn

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
#from google.colab.patches import cv2_imshow

In [ ]:
det=MTCNN()

In [ ]:
img=cv2.imread('../input/outputofnoout/new-outputs-here/0blue-mask.jpg')

In [ ]:
model=load_model('../input/imp-model-for/mymodel.h5')

In [ ]:
k=det.detect_faces(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
for i in range(0,np.array(k).shape[0]):
    x1,y1,x2,y2=k[i]['box']
    x1=max(0,x1)
    y1=max(0,y1)
    x2=x2+x1
    y2=y2+y1
    face=img[y1:y2,x1:x2]
    face=cv2.resize(face,(128,128))
    face=preprocess_input(face)
    face=np.expand_dims(face,axis=0)
    pred=model.predict(face)
    pred=np.argmax(pred,axis=1)
    print(pred)

In [ ]:
preds=[]
real_label=[]
pred_in_pred=[]

In [ ]:
det=MTCNN()

In [ ]:
for i in paths.list_images('../input/3classmodelnewevaluation/new_dataset/'):
    label= i.split(os.path.sep)[-2]
    real_label.append(label)
    image=cv2.imread(i)
    img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    final_det=det.detect_faces(img)
    if len(final_det)==0:
        pred_in_pred.append("not readable")
        continue
    else:
        
        x1,y1,w,h=final_det[0]['box']
        x2=x1+w
        y2=y1+h
        if x1<0  or y1 < 0:
            pred_in_pred.append("negative label")
            continue
            
        else:
            face=img[y1:y2,x1:x2]
            face=cv2.resize(face,(128,128))
            face=img_to_array(face)
            face=preprocess_input(face)
            face=np.expand_dims(face,axis=0)
            smallpred=model.predict(face)
            smallpred=np.argmax(smallpred,axis=1)
            pred_in_pred.append(smallpred)
        
        

        
        
            

In [ ]:
k=list(paths.list_images('../input/3classmodelnewevaluation/new_dataset/'))

In [ ]:
for i in list(paths.list_images('../input/3classmodelnewevaluation/new_dataset/'))[:1254]:
    label= i.split(os.path.sep)[-2]
    real_label.append(label)
    image=cv2.imread(i)
    img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    final_det=det.detect_faces(img)
    if len(final_det)==0:
        pred_in_pred.append("not readable")
        
    else:
        
        x1,y1,w,h=final_det[0]['box']
        x1=max(0,x1)
        y1=max(0,y1)
        x2=x1+w
        y2=y1+h
            
        face=image[y1:y2,x1:x2]
        face=cv2.resize(face,(128,128))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        smallpred=model.predict(face)
        smallpred=np.argmax(smallpred,axis=1)
        pred_in_pred.append(smallpred)
        
        

        
        
            

In [ ]:
real_label=np.array(real_label)

In [ ]:
le=LabelEncoder()
real_label=le.fit_transform(real_label)
real_label=to_categorical(real_label)

In [ ]:
pred_in_pred=np.array(pred_in_pred)

In [ ]:
qw=np.where(pred_in_pred=='negative label')
qw

In [ ]:
qw=np.array(qw)
qw.shape

In [ ]:
pt=np.where(pred_in_pred=='not readable')
pt

In [ ]:
pt=np.array(pt)
pt.shape

In [ ]:
preed=np.delete(preed,(np.where(preed=='not readable') or  np.where(preed=='negative label')))

In [ ]:
preed.shape


In [ ]:
real_new=np.argmax(real_label,axis=1)

In [ ]:
real_new=np.delete(real_new,(np.where(pred_in_pred=='not readable') or np.where(pred_in_pred=='negative label')))

In [ ]:
real_new=np.delete(real_new,np.where(pred_in_pred=='negative label'))

In [ ]:
real_new.shape

In [ ]:
preed

In [ ]:
code=[]

In [ ]:
for i in range(0,len(preed)):
    code.append(preed[i][0])

In [ ]:
print(classification_report(real_new[:1199],code,target_names=lb.classes_))

In [ ]:
code=np.array(code)

In [ ]:
k=np.where(pre=='negative label')

In [ ]:
pre.shape


In [ ]:
!cd

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))
# serialize the model to disk
print("[INFO] saving mask detector model...")
tf.keras.models.save_model(model=model,filepath='./mymodel.h5',save_format='h5')

In [ ]:
import cv2

In [ ]:
from matplotlib.pyplot import imread,imshow
k=imread('../input/3classmodelnewevaluation/new_dataset/with_mask/115blue-mask.jpg')
k=cv2.resize(k,(128,128))
k=img_to_array(k)
k=preprocess_input(k)
k=np.expand_dims(k,axis=0)
model.predict(k)[0]

In [ ]:
z=imshow(imread('../input/outputofnoout/new-outputs-here/115blue-mask.jpg'))

In [ ]:
MASKCONFIDENCE = 0.2
FACECONFIDENCE = 0.5
myPath = '../input/facemask/face-mask-detector/examples/myexamples/profilepic4.jpg'

In [ ]:
"""# USAGE
# python detect_mask_image.py --image examples/example_01.png

# import the necessary packages
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import random
# construct the argument parser and parse the arguments
""""""ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-f", "--face", type=str,
	default="face_detector",
	help="path to face detector model directory")
ap.add_argument("-m", "--model", type=str,
	default="mask_detector.model",
	help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections")
args = vars(ap.parse_args())""""""

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join(['../input/facemask/face-mask-detector/face_detector', "deploy.prototxt"])
weightsPath = os.path.sep.join(['../input/facemask/face-mask-detector/face_detector',
	"res10_300x300_ssd_iter_140000.caffemodel"])

print(prototxtPath,type(prototxtPath))

net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model =load_model('/kaggle/working/mask_detector_model/')

# load the input image from disk, clone it, and grab the image spatial
# dimensions

######################################################################
choser = random.randrange(2)
imageid = random.randrange(100)

chosepath = '/maksssksksss'+str(imageid)+'.png'

# if choser == 1: chosepath = 'with_mask/'+str(imageid)+'-with-mask.jpg'
# else : chosepath = 'without_mask/'+str(imageid)+'.jpg'
  
"""
##../input/facemask/face-mask-detector/dataset/
##myPath = '../input/facemask/face-mask-detector/examples/images'+chosepath
"""

image_path = '../input/facemask/face-mask-detector/examples/example_03.png'
print(image_path)

image = cv2.imread(image_path)
print(image.shape)
orig = image.copy()
(h, w) = image.shape[:2]

#######################################################################

# construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with
	# the detection
	confidence = detections[0, 0, i, 2]

	# filter out weak detections by ensuring the confidence is
	# greater than the minimum confidence
	if confidence > MASKCONFIDENCE:
		# compute the (x, y)-coordinates of the bounding box for
		# the object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		# ensure the bounding boxes fall within the dimensions of
		# the frame
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

		# extract the face ROI, convert it from BGR to RGB channel
		# ordering, resize it to 224x224, and preprocess it
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)

		# pass the face through the model to determine if the face
		# has a mask or not
		(mask, withoutMask) = model.predict(face)[0]

		# determine the class label and color we'll use to draw
		# the bounding box and text
#######################################################################################
		if(mask>FACECONFIDENCE or withoutMask>FACECONFIDENCE):
			label = "Mask" if mask > withoutMask else "No Mask"
			color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
#######################################################################################
		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(image, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)


"""

## facenet-pytorch

In [50]:
#!pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0
#import torch
#torch.__version__
#torch.cuda.set_per_process_memory_fraction(0.8, device=0)
#!pip install imutils
from imutils import paths
##import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import preprocess_input
#from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

#from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os
#from google.colab.patches import cv2_imshow
!conda  install  -y gdown

!pip install facenet-pytorch
from facenet_pytorch import MTCNN
!gdown --id 1-C2ND6AJp98ff4a_BF1caIxnvM82aXii
import zipfile
with zipfile.ZipFile('./new_data.zip', 'r') as zip_ref:
    zip_ref.extractall('./new')
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('')
mtcnn=MTCNN(select_largest=True,device='cuda')
real_label=[]
pred_in_pred=[]
#model=tf.keras.models.load_model('../input/imp-model-for/mymodel.h5')
for i in list(paths.list_images('./new')):
    
    label= i.split(os.path.sep)[-2]
    real_label.append(label)
    image=cv2.imread(i)
    img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    box,prob=mtcnn.detect(img)
    if prob[0]==None:
        pred_in_pred.append("not readable")
    else:
        x1,y1,x2,y2=box[0][:]
        x1=int(max(0,x1))
        y1=int(max(0,y1))
        x2=int(max(0,x2))
        y2=int(max(0,y2))
        face=image[x1:x2,y1:y2]
        if face.shape[0]==0 or face.shape[1]==0:
            pred_in_pred.append("not readable")
            continue
        else:
            
            face=cv2.resize(face,(128,128))
            face=preprocess_input(face)
            face=np.expand_dims(face,axis=0)
            
            smallpred=model1.predict(face)[0]
            pred_in_pred.append(smallpred)
len(pred_in_pred)
pred_in_pred=np.array(pred_in_pred)
preed=pred_in_pred
preed=np.delete(preed,np.where(pred_in_pred=='not readable'))
real_label=np.array(real_label)
real_new=real_label
real_new=np.delete(real_new,np.where(pred_in_pred=='not readable'))
len(real_new),len(preed)
le=LabelEncoder()
real_new=le.fit_transform(real_new)
real_new=to_categorical(real_new)
c1=[]
for i in range(len(preed)):
    c1.append(preed[i][:])
print(classification_report(np.argmax(real_new,axis=1),np.argmax(c1,axis=1),target_names=le.classes_))

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_0         3.1 MB  conda-forge
    gdown-3.13.0               |     pyhd8ed1ab_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  gdown              conda-forge/noarch::gdown-3.13.0-pyhd8ed1ab_0

The following packages will be UPDATED:

  ca-certificates                      2020.6.20-hecda079_0 --> 2021.5.30-ha878542_0
  certifi                          2020.6.20-py37h

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


##### 